In [18]:
from ntk import empirical_ntk_ntk_vps, empirical_ntk_jacobian_contraction
from NeuralNet import MTRAgent, ReinforceAgent
from MtrNet import MtrNet
import torch
from original import OriginalMTR
from torch.func import functional_call, vmap, vjp, jvp, jacrev
GAME_NAME = "Toy"
"""
            INITIALIZING AGENTS and ENVIRONMENT
"""
"""-------------------------------------------------------------------"""
from enviroment import *
env, obs_dim, action_dim = game_setup(GAME_NAME )

In [19]:
"""
                 SETUP AGENT and TRAINING
"""
"""REINFORCE"""
BETA_REIN_FULL = 0.1 #Betas are scalar by which we multiply bias terms in NN at initialization. They are used to control Chaos order.
BETA_REIN_RESNET = BETA_REIN_FULL
LEARNING_RATE_REIN = 0.0001
HORIZON_REIN = 4
HIDDEN_DIM_REIN = 32
"""MATRYOSHKA"""
BETA_MTR_FULL = 0.1
LEARNING_RATE_MTR = 0.0001
HORIZON_MTR= 4
HIDDEN_DIM_MTR = 32
TAU = 1
"""ResNet"""
BETA_MTR_RESNET = BETA_MTR_FULL
HIDDEN_DIM_RESNET = 16
LEARNING_RATE_RESNET = LEARNING_RATE_MTR * 10
"""Original"""
HIDDEN_DIM_ORIGINAL = 16
HORIZON_ORIGINAL = 3
BETA_ORIGINAL = 0.1
TAU_ORIGINAL = 0.4
LEARNING_RATE_ORIGINAL = LEARNING_RATE_RESNET
"""Training"""
NUM_EPOCHES = 500
N_EPISODES = 10 # number of episodes per epoch. Used for both: Reinforce and Matryoshka
"""
                    INITIALIZING AGENTS
"""
"""-------------------------------------------------------------------"""
from NeuralNet import ReinforceAgent, MTRAgent
from MtrNet import ReinforceMtrNetAgent, MtrNetAgent
import torch

#Reinforce Full Connected
agent1 = ReinforceAgent(obs_dim,action_dim, hidden_dim=HIDDEN_DIM_REIN,horizon= HORIZON_REIN, learning_rate= LEARNING_RATE_REIN, game_name=GAME_NAME, beta = BETA_MTR_FULL)
agent1.policy.ntk_init(beta=agent1.beta)
#Reinforce MTRNet (Custom ResNet)
agent2 = ReinforceMtrNetAgent(obs_dim,action_dim, hidden_dim=HIDDEN_DIM_RESNET,horizon= HORIZON_REIN, learning_rate= LEARNING_RATE_RESNET , game_name= GAME_NAME, tau=TAU, beta=BETA_REIN_RESNET)
agent2.policy.ntk_init(beta=agent2.beta)
#MTR extra-dimension
agent3 = MTRAgent(obs_dim, action_dim, hidden_dim= HIDDEN_DIM_MTR , horizon= HORIZON_MTR, learning_rate= LEARNING_RATE_MTR, game_name= GAME_NAME, tau=TAU, beta=BETA_MTR_FULL)
agent3.policy.ntk_init(beta=agent3.beta)
#MTR with MTRNet
agent4 = MtrNetAgent(obs_dim,action_dim, hidden_dim=HIDDEN_DIM_RESNET,horizon= HORIZON_MTR, learning_rate= LEARNING_RATE_RESNET , game_name= GAME_NAME, tau=TAU, beta=BETA_MTR_RESNET)
agent4.policy.ntk_init(beta=agent4.beta)
from original import OriginalMtrAgent
# Set-up agent
agent5 = OriginalMtrAgent(obs_dim,action_dim, hidden_dim=HIDDEN_DIM_ORIGINAL,horizon= HORIZON_ORIGINAL, learning_rate= LEARNING_RATE_ORIGINAL , game_name= GAME_NAME, tau=TAU_ORIGINAL, beta=BETA_ORIGINAL)
agent5.policy.ntk_init(beta=agent5.beta)
Agents = [agent2, agent4, agent5]

tensor([[[ 0.3543, -0.3543],
         [-0.3543,  0.3543]]])

In [35]:
from utils import run_episodes_mtr
episodes = run_episodes_mtr(agent3, env,n_episodes=1)
states, _, _, _, _ = zip(*episodes[0])
x1 = states[0]
x2 = states[1]
print("States")
print("x1:",x1)
print("x2:", x2)
print("")
print(agent3.name)
print(agent3.ntk(x1,x2))
x1 = x1[:-1]
x2 = x2[:-1]
print(agent1.name)
print(agent1.ntk(x1,x2))
step = 1
for agent in Agents:
    print(agent.name)
    print(agent.ntk(x1,x2, step))

States
x1: [1.   0.   0.   0.   0.   0.   0.   0.   0.   0.75]
x2: [0.  1.  0.  0.  0.  0.  0.  0.  0.  0.5]

MTR
tensor([[[ 0.4352, -0.4352],
         [-0.4352,  0.4352]]])
REIN
tensor([[[ 0.2093, -0.2093],
         [-0.2093,  0.2093]]])
ReinMtrNet
tensor([[[ 0.6880, -0.6880],
         [-0.6880,  0.6880]]])
MtrNet
tensor([[[ 0.7034, -0.7034],
         [-0.7034,  0.7034]]])
OriginalMtr
tensor([[[ 0.7753, -0.7753],
         [-0.7753,  0.7753]]])


In [ ]:
"""
        TRAINING (REINFORCE) PARALLEL
        Remark: NOT WORKING, in process...
"""

LEARNING_RATE = 0.01
NUM_EPOCHES = 30
N_EPISODES = 20 # ^number of episodes per epoch
"""-------------------------------------------------------------------"""
import torch.optim as optim
from try_grad_policy import train_policy
import multiprocessing

optimizer = optim.Adam(policy.parameters(), lr=LEARNING_RATE)  # Define the optimizer
optimizer_ntk = optim.Adam(ntk_policy.parameters(), lr=LEARNING_RATE)  # Define the optimizer


initialize_env(env)

output_queue1 = multiprocessing.Queue()
output_queue2 = multiprocessing.Queue()
run_process1 = multiprocessing.Process(target=run_episodes, args=(policy, env,output_queue1, N_EPISODES))
run_process2 = multiprocessing.Process(target=run_episodes, args=(ntk_policy, env,output_queue2, N_EPISODES))


# Train for a number of epochs
for epoch in range(NUM_EPOCHES ):
    run_process1.start()  # Collect episodes
    run_process2.start()
    # Join the processes to wait for them to finish
    run_process1.join()
    run_process2.join()
    # Get the outputs from both processes
    episodes = output_queue1.get()
    ntk_episodes = output_queue2.get()
    loss = train_policy(policy, optimizer, episodes)  # Update the policy based on the episodes
    ntk_loss = train_policy(ntk_policy, optimizer_ntk, ntk_episodes)  # Update the ntk_policy based on the episodes
    print(f'Episode {epoch + 1}, Total Loss: {BETA1}={loss} and {BETA2}={ntk_loss} ')
close_env(env)
"-------------------TEST-------------------"
env = gym.make('CartPole-v1', render_mode = "human" )
run_episodes(policy, env, n_episodes=10)